<h1>Accessing data though 3D-Beacons API</h1>

<img src="https://raw.githubusercontent.com/3D-Beacons/3D-Beacons/main/assets/3D-Beacons-logo.png" height="100" align="right">

## Introduction

Welcome to this Google Colab notebook, an essential companion to our paper on accessing data through the 3D-Beacons API.

This notebook serves as a practical resource for researchers, developers, and data enthusiasts who are interested in exploring the potential of the 3D-Beacons API and leveraging its capabilities. By following this guide, you will gain a deep understanding of how to interact with the 3D-Beacons API to access and analyze various types of data.
To supplement your learning, we have provided links to the full paper as well as documentation resources that will assist you in navigating the API effectively.

Documentation Link: [3D-Beacons API Documentation](https://www.ebi.ac.uk/pdbe/pdbe-kb/3dbeacons/api/#/default/get_uniprot_summary_uniprot_summary__qualifier__json_get)

<br>

**Further readings:**

**3D-Beacons: Decreasing the gap between protein sequences and structures through a federated network of protein structure data resources**

Mihaly Varadi, Sreenath Nair, Ian Sillitoe, Gerardo Tauriello, Stephen Anyango, Stefan Bienert, Clemente Borges, Mandar Deshpande, Tim Green, Demis Hassabis, Andras Hatos, Tamas Hegedus, Maarten L Hekkelman, Robbie Joosten, John Jumper, Agata Laydon, Dmitry Molodenskiy, Damiano Piovesan, Edoardo Salladini, Steven L. Salzberg, Markus J Sommer, Martin Steinegger, Erzsebet Suhajda, Dmitri Svergun, Luiggi Tenorio-Ku, Silvio Tosatto, Kathryn Tunyasuvunakool, Andrew Mark Waterhouse, Augustin Žídek, Torsten Schwede, Christine Orengo, Sameer Velankar
3 August 2022; BioRxiv https://doi.org/10.1101/2022.08.01.501973


  ## Instructions <a name="INSTRUCTIONS"></a>

* Quick Start <a name="Quick Start"></a>

In order to make the learning experience more accessible and interactive, we have incorporated widgets that allow you to provide inputs and customize certain aspects of the code. These widgets serve as interactive elements that enhance your ability to interact with the code and observe the impact of different inputs on the results. Throughout this tutorial, you will come across code chunks that generate these interactive widgets.

  * How to use Google Colab <a name="Google Colab"></a>
1. To run a code cell, click on the cell to select it. You will notice a play button (▶️) on the left side of the cell. Click on the play button or press Shift+Enter to run the code in the selected cell.
2. The code will start executing, and you will see the output, if any, displayed below the code cell.
3. Move to the next code cell and repeat steps 2 and 3 until you have executed all the desired code cells in sequence.
4. The currently running step is indicated by a circle with a stop sign next to it.
If you need to stop or interrupt the execution of a code cell, you can click on the stop button (■) located next to the play button.


*Remember to run the code cells in the correct order, as their execution might depend on variables or functions defined in previous cells. You can modify the code in a code cell and re-run it to see updated results.*

*Note: If the notebook runtime is restarted you will need to re-run the first 3 code chunks located in the Set Up section.*



## Contact us

Requests for clarifications or reporting bugs can be made via pdbekb_help@ebi.ac.uk.






---



## &nbsp; Set Up

**Run the next 2 code blocks in the specific order to:**
1. Install all neccessary libraries.
2. Import libraries and prepare functions for easier functionality.
3. Mount drive to download files.



In [ ]:
#@title Install libraries
!pip install ijson gwpy &> /dev/null

print("Succesfully installed!")

Succesfully installed!


After running the next code, you may be prompted to grant access to your Google Drive. This is necessary for Google Colab to download the files and save them to your Drive.
<br>
<br>
Please follow the on-screen instructions to provide the necessary permissions, as it enables seamless integration between Colab and your Drive. Rest assured that your data and files are secure and will not be accessed without your explicit permission.
<br>

In [ ]:
#@title Mount Drive to download files
######## LIBRARIES
import ijson
import requests, sys, json
import ipywidgets as wgt
from urllib.request import urlopen
# Importing the necessary libraries for mounting Google Drive
import os
from google.colab import drive

######## GOOGLE DRIVE
# Mounting the Google Drive to access files and directories
drive.mount('/content/drive')
destination_path = '/content/drive/MyDrive/3D_Beacons_files'

# Check whether the specified path exists or not
isExist = os.path.exists(destination_path)
if not isExist:

   # Create a new directory because it does not exist
   os.makedirs(destination_path)
   print("The new directory is created!")

######## FUNCTIONS
#defining functions for search and download
def Search3DBeacons(ID):
  WEBSITE_API = "https://www.ebi.ac.uk/pdbe/pdbe-kb/3dbeacons/api/uniprot/summary/"

  r = ijson.parse(urlopen(f"{WEBSITE_API}{ID}.json"))
  structures = list(ijson.items(r, "structures.item", use_float=True))
  return structures

# Function to download a file from a given URL and save it to the Google Drive
def download_file(url):
  os.chdir(destination_path)
  !wget "$url"

# Defining function for sequence search
def sequence_search(sequence):
    global job_id

    post_url = "https://wwwdev.ebi.ac.uk/pdbe/pdbe-kb/3dbeacons/api/sequence/search"
    query_sequence = {"sequence": sequence}

    try:
      #send POST request to perform sequence search
        response = requests.post(post_url, json=query_sequence)
        response.raise_for_status() #Raise an exception if it fails
        if response.status_code == 200:
            print("Your search was successful")
            job_id = response.json()["job_id"]
        else:
            print(f"Request failed with status code {response.status_code}")
            exit()
    except requests.RequestException as e:
        print(f"An error occurred during the POST request: {e}")
        exit()

    retrieve_results(job_id)

def retrieve_results(job_id):
    get_url = "https://wwwdev.ebi.ac.uk/pdbe/pdbe-kb/3dbeacons/api/sequence/result"
    try:
      #send GET request to retrieve the results from the search
        response = urlopen(f"{get_url}?job_id={job_id}")
        r = ijson.items(response, 'item')
        for item in r:
            print(item)
    except Exception as e:
        print(f"An error occurred during the GET request: {e}")
        exit()

# Defining function for Ensembl search
def ensembl_search(ENSEMBL_ID):
    url = "https://wwwdev.ebi.ac.uk/pdbe/pdbe-kb/3dbeacons/api/ensembl/summary/" #url

    try:
        # Send GET request to retrieve Ensembl summary
        response = urlopen(f"{url}{ENSEMBL_ID}.json")
        if response.status == 200:
            print("Ensembl search successful")
        else:
            print(f"Ensembl search failed with status code {response.status}")

        r = ijson.parse(response)
        ensembls = ijson.items(r, "uniprot_mappings.item", use_float=True)

        # Loop for better printing
        for ensembl in ensembls:
            print(ensembl)
    except Exception as e:
        print(f"An error occurred during the Ensembl search: {e}")
        exit()



######## WIDGETS
#Defining UniProt ID
Uniprot_ID = wgt.Textarea(
    value= "",
    placeholder='Uniprot ID',
    description='Enter an UniProt ID:',
    style={'description_width': 'initial'},
    disabled=False
)

#Defining all model type
model_type = wgt.Textarea(
    value="",
    placeholder='Enter model category',
    description='Enter a model type:',
    style={'description_width': 'initial'},
    disabled=False
)

#create a widget for provider
provider_value = wgt.Textarea(
    value="",
    placeholder='Enter provider',
    description='Enter a provider:',
    style={'description_width': 'initial'},
    disabled=False
)

#create a widget for resolution
resolution_filter = wgt.FloatSlider(
    value=2.0,
    min=0,
    max=5.0,
    step=0.1,
    description='Resolution:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

#create a widget to download specific model
model_download = wgt.Textarea(
    value="",
    placeholder='Enter model identifier',
    description='Model identifier:',
    style={'description_width': 'initial'},
    disabled=False
)

#Defining Sequence
AA_search = wgt.Textarea(
    value= "",
    placeholder='AA Sequence',
    description='Enter AA Sequence:',
    style={'description_width': 'initial'},
    layout = wgt.Layout(width='800px'),
    disabled=False
)

#ENSEMBL_ID
EnsemblID_serch = wgt.Textarea(
    value= "",
    placeholder='ENSEMBL ID',
    description='Enter Ensembl ID',
    style={'description_width': 'initial'},
    disabled=False
)

######## API
#url_API
WEBSITE_API = "https://www.ebi.ac.uk/pdbe/pdbe-kb/3dbeacons/api/uniprot/summary/"

Mounted at /content/drive




---



##1.&nbsp; BASIC SEARCH

This section implements API to search and filter predicted and experimentally determined structures available from the 3D-Beacons network.

Once you have found a model, you can download the specific

### 1.1.&nbsp;  Get all macromolecular structures for a single entry

  On this sub-section you will be able to retrieve all available structures and metainformation in 3D-Beacons from a single Uniprot accession ID.

In [ ]:
#Run this code to display the widget
display(Uniprot_ID)

Textarea(value='', description='Enter an UniProt ID:', placeholder='Uniprot ID', style=DescriptionStyle(descri…

In [ ]:
#Run this code to search the network using the input from the widget
structures = Search3DBeacons(Uniprot_ID.value)

#print available structures from different providers
for structure in structures:
    print(structure)

{'summary': {'model_identifier': '3d06', 'model_category': 'EXPERIMENTALLY DETERMINED', 'model_url': 'https://www.ebi.ac.uk/pdbe/static/entry/3d06_updated.cif', 'model_format': 'MMCIF', 'model_type': None, 'model_page_url': 'https://www.ebi.ac.uk/pdbe/entry/pdb/3d06', 'provider': 'PDBe', 'number_of_conformers': None, 'ensemble_sample_url': None, 'ensemble_sample_format': None, 'created': '2008-05-01', 'sequence_identity': Decimal('99.0'), 'uniprot_start': 94, 'uniprot_end': 293, 'coverage': Decimal('0.509'), 'experimental_method': 'X-RAY DIFFRACTION', 'resolution': Decimal('1.2'), 'confidence_type': None, 'confidence_version': None, 'confidence_avg_local_score': None, 'oligomeric_state': None, 'preferred_assembly_id': None, 'entities': [{'entity_type': 'POLYMER', 'entity_poly_type': 'POLYPEPTIDE(L)', 'identifier': 'P04637', 'identifier_category': 'UNIPROT', 'description': 'Cellular tumor antigen p53', 'chain_ids': ['A']}, {'entity_type': 'NON-POLYMER', 'entity_poly_type': None, 'identi

### 1.2.&nbsp;  Performing one  filter


#### 1.2.1.&nbsp; Filter by Model type
Results can be filtered according to the model category in the 3D-Beacons Network.
Models are classified as:

* EXPERIMENTALLY DETERMINED
* CONFORMATIONAL ENSEMBLE
* TEMPLATE-BASED
* AB-INITIO

Code below allows you to filter the previous search using a model category.

In [ ]:
#Run this code to display a widget for input
display(model_type) #display widget

Textarea(value='', description='Enter a model type:', placeholder='Enter model category', style=DescriptionSty…

In [ ]:
#Run this code to filter according to model category
for structure in structures:
    model_category = structure.get('summary', {}).get('model_category')
    if model_category == model_type.value:
        print(structure)

{'summary': {'model_identifier': '3d06', 'model_category': 'EXPERIMENTALLY DETERMINED', 'model_url': 'https://www.ebi.ac.uk/pdbe/static/entry/3d06_updated.cif', 'model_format': 'MMCIF', 'model_type': None, 'model_page_url': 'https://www.ebi.ac.uk/pdbe/entry/pdb/3d06', 'provider': 'PDBe', 'number_of_conformers': None, 'ensemble_sample_url': None, 'ensemble_sample_format': None, 'created': '2008-05-01', 'sequence_identity': Decimal('99.0'), 'uniprot_start': 94, 'uniprot_end': 293, 'coverage': Decimal('0.509'), 'experimental_method': 'X-RAY DIFFRACTION', 'resolution': Decimal('1.2'), 'confidence_type': None, 'confidence_version': None, 'confidence_avg_local_score': None, 'oligomeric_state': None, 'preferred_assembly_id': None, 'entities': [{'entity_type': 'POLYMER', 'entity_poly_type': 'POLYPEPTIDE(L)', 'identifier': 'P04637', 'identifier_category': 'UNIPROT', 'description': 'Cellular tumor antigen p53', 'chain_ids': ['A']}, {'entity_type': 'NON-POLYMER', 'entity_poly_type': None, 'identi


#### 1.2.2.&nbsp; Filter by Provider

The 3D-Beacons Network facilitates the aggregation of coordinate files and metadata for both experimental and theoretical protein models. It encompasses a wide range of state-of-the-art and specialized model providers, as well as data from the European Protein Data Bank (PDBe).

Model providers:
* PDBe
* SWISS-MODEL
* AlphaFold DB
* Genome3D
* SASBDB
* AlphaFill
* ModelArchive
* Protein Ensemble Database (PED)

For further information, visit the [Documentation](https://wwwdev.ebi.ac.uk/pdbe/pdbe-kb/3dbeacons/docs) to check all the current providers.

In [ ]:
#Run this code to display widget
display(provider_value)

Textarea(value='AlphaFold DB', description='Enter a provider:', placeholder='Enter  provider', style=Descripti…

In [ ]:
#Run this code to filter results according to given provider
for structure in structures:
    provider = structure.get('summary', {}).get('provider')

    if provider == provider_value.value:
        print(structure)

{'summary': {'model_identifier': 'AF-P04637-F1', 'model_category': 'AB-INITIO', 'model_url': 'https://alphafold.ebi.ac.uk/files/AF-P04637-F1-model_v4.cif', 'model_format': 'MMCIF', 'model_type': None, 'model_page_url': 'https://alphafold.ebi.ac.uk/entry/P04637', 'provider': 'AlphaFold DB', 'number_of_conformers': None, 'ensemble_sample_url': None, 'ensemble_sample_format': None, 'created': '2022-06-01', 'sequence_identity': Decimal('1.0'), 'uniprot_start': 1, 'uniprot_end': 393, 'coverage': Decimal('1.0'), 'experimental_method': None, 'resolution': None, 'confidence_type': 'pLDDT', 'confidence_version': None, 'confidence_avg_local_score': Decimal('74.33'), 'oligomeric_state': None, 'preferred_assembly_id': None, 'entities': [{'entity_type': 'POLYMER', 'entity_poly_type': 'POLYPEPTIDE(L)', 'identifier': 'P04637', 'identifier_category': 'UNIPROT', 'description': 'Cellular tumor antigen p53', 'chain_ids': ['A']}]}}


### 1.3.&nbsp;  Performing two filters

This section uses two sets of filters to display the models available in the network

#### 1.3.1&nbsp;  Filter by Model type and Coverage

The code below will filter all the structures from a Uniprot ID and will save the model with the highest coverage to Google Drive.

In [ ]:
#Run this code to display widgets
display(Uniprot_ID) #display widget for UniProt ID
display(model_type)

Textarea(value='P04637', description='Enter an Uniprot ID:', placeholder='Uniprot ID', style=DescriptionStyle(…

Textarea(value='EXPERIMENTALLY DETERMINED', description='Enter a model type:', placeholder='Enter model catego…

In [ ]:
#Run this code to perform search and filter
response = urlopen(f"{WEBSITE_API}{Uniprot_ID.value}.json")
r = ijson.parse(response)
structures = list(ijson.items(r, "structures.item", use_float=True))
structures.sort(key=lambda x: x.get('summary', {}).get('coverage', 0), reverse=True)

highest_coverage_structure = None
for structure in structures:
    model_category = structure.get('summary', {}).get('model_category')
    if model_category == model_type.value:
        highest_coverage_structure = structure
        break

if highest_coverage_structure is not None:
    print(highest_coverage_structure)

model_download = highest_coverage_structure.get('summary', {}).get('model_identifier')
for structure in structures:
    model = structure.get('summary', {}).get('model_identifier')
    if model == model_download:
        model_url = structure.get("summary", {}).get("model_url")
        download_file(model_url)


{'summary': {'model_identifier': '7xzz', 'model_category': 'EXPERIMENTALLY DETERMINED', 'model_url': 'https://www.ebi.ac.uk/pdbe/static/entry/7xzz_updated.cif', 'model_format': 'MMCIF', 'model_type': None, 'model_page_url': 'https://www.ebi.ac.uk/pdbe/entry/pdb/7xzz', 'provider': 'PDBe', 'number_of_conformers': None, 'ensemble_sample_url': None, 'ensemble_sample_format': None, 'created': '2022-06-03', 'sequence_identity': Decimal('100.0'), 'uniprot_start': 1, 'uniprot_end': 393, 'coverage': Decimal('1.0'), 'experimental_method': 'ELECTRON MICROSCOPY', 'resolution': Decimal('4.07'), 'confidence_type': None, 'confidence_version': None, 'confidence_avg_local_score': None, 'oligomeric_state': None, 'preferred_assembly_id': None, 'entities': [{'entity_type': 'POLYMER', 'entity_poly_type': 'POLYPEPTIDE(L)', 'identifier': 'P04637', 'identifier_category': 'UNIPROT', 'description': 'Cellular tumor antigen p53', 'chain_ids': ['N', 'L', 'K', 'M']}, {'entity_type': 'POLYMER', 'entity_poly_type': '

#### 1.3.2&nbsp;  Filter by model type and retrieve results according to resolution.


The code below will filter all the structures from a Uniprot ID and will save the models with a resolution higher than the specified on the widget.

In [ ]:
#Run this code to display widgets
display(Uniprot_ID)
display(provider_value)
display(resolution_filter)

Textarea(value='', description='Enter an Uniprot ID:', placeholder='Uniprot ID', style=DescriptionStyle(descri…

Textarea(value='', description='Enter a provider:', placeholder='Enter  provider', style=DescriptionStyle(desc…

FloatSlider(value=2.0, continuous_update=False, description='Resolution:', max=5.0, readout_format='.1f')

In [ ]:
#Run this code to search and filter results according to the input provided
r = ijson.parse(urlopen(f"{WEBSITE_API}{Uniprot_ID.value}.json")) #build
structures = list(ijson.items(r, "structures.item", use_float=True))

#filter for experimental structures
high_resolution_structures = []
for structure in structures:
    provider = structure.get('summary', {}).get('provider')
    resolution = structure.get('summary', {}).get('resolution')

    if provider == provider_value.value and resolution is not None and resolution < resolution_filter.value:
        # Append the structure to the list without assigning the result back to the list
        high_resolution_structures.append(structure)

# Perform filter with the high-resolution structures
for structure in high_resolution_structures:
    print(structure)

{'summary': {'model_identifier': '3d06', 'model_category': 'EXPERIMENTALLY DETERMINED', 'model_url': 'https://www.ebi.ac.uk/pdbe/static/entry/3d06_updated.cif', 'model_format': 'MMCIF', 'model_type': None, 'model_page_url': 'https://www.ebi.ac.uk/pdbe/entry/pdb/3d06', 'provider': 'PDBe', 'number_of_conformers': None, 'ensemble_sample_url': None, 'ensemble_sample_format': None, 'created': '2008-05-01', 'sequence_identity': Decimal('99.0'), 'uniprot_start': 94, 'uniprot_end': 293, 'coverage': Decimal('0.509'), 'experimental_method': 'X-RAY DIFFRACTION', 'resolution': Decimal('1.2'), 'confidence_type': None, 'confidence_version': None, 'confidence_avg_local_score': None, 'oligomeric_state': None, 'preferred_assembly_id': None, 'entities': [{'entity_type': 'POLYMER', 'entity_poly_type': 'POLYPEPTIDE(L)', 'identifier': 'P04637', 'identifier_category': 'UNIPROT', 'description': 'Cellular tumor antigen p53', 'chain_ids': ['A']}, {'entity_type': 'NON-POLYMER', 'entity_poly_type': None, 'identi

In [ ]:
# Run this code to download the structures with resolution higher than the previous selction on the widge
for structure in high_resolution_structures:
    model_url = structure.get("summary", {}).get("model_url")
    # Use wget or other methods to download the files
    download_file(model_url)
    print("Downloading:", model_url)



---



##2.&nbsp; SEQUENCE-BASED SEARCH

The 3D-Beacons Network has introduced sequence similarity search functionality which allows you to query the network using the amino acid sequence of a protein.

The Sequence Similarity Search option available through the network uses the Basic Local Alignment Search Tool (BLAST, Altschul et al., 1990) to find regions of sequence similarity by aligning them with a query sequence. By evaluating the match between the network and query sequence, valuable insights into the structure, function, and evolutionary aspects can be obtained, thus facilitating targeted and systematic exploration of protein structures.

The code presented below allows you to search the network by performing a sequence-based search via API.

In [ ]:
#Run this code to display widget to insert AA as query
display(AA_search)

Textarea(value='', description='Enter AA Sequence:', layout=Layout(width='800px'), placeholder='AA Sequence', …

In [ ]:
#Run this code to perform sequence search, it will use the AA from the widget
sequence_search(AA_search.value)

Your search was successful
{'accession': 'A0A452IEQ2', 'id': 'A0A452IEQ2_9SAUR', 'description': 'Uncharacterized protein OS=Gopherus agassizii OX=38772 PE=4 SV=1', 'hit_length': 308, 'hit_hsps': [{'hsp_score': Decimal('296.0'), 'hsp_bit_score': Decimal('118.627'), 'hsp_align_len': 50, 'hsp_identity': Decimal('100.0'), 'hsp_positive': Decimal('100.0'), 'hsp_qseq': 'RVKALRWQCIECKTCSSCRDQGKNADNMLFCDSCDRGFHMECCDPPLTRM', 'hsp_hseq': 'RVKALRWQCIECKTCSSCRDQGKNADNMLFCDSCDRGFHMECCDPPLTRM', 'hsp_mseq': 'RVKALRWQCIECKTCSSCRDQGKNADNMLFCDSCDRGFHMECCDPPLTRM', 'hsp_expect': Decimal('4.4E-31')}], 'summary': {'uniprot_entry': {'ac': 'A0A452IEQ2', 'id': 'A0A452IEQ2_9SAUR', 'uniprot_checksum': '4A1FD79F38685083', 'sequence_length': 308, 'segment_start': 1, 'segment_end': 308}, 'structures': [{'summary': {'model_identifier': 'AF-A0A452IEQ2-F1', 'model_category': 'AB-INITIO', 'model_url': 'https://alphafold.ebi.ac.uk/files/AF-A0A452IEQ2-F1-model_v4.cif', 'model_format': 'MMCIF', 'model_type': None, 'model_



---



##3.&nbsp; RETRIEVING ENSEMBL SUMMARY VIA 3D-BEACONS

The 3D-Beacons network provides the resource for retrieving Ensembl summaries to explore the diverse transcript variants associated with a given gene. ENSEMBL provides information about the different transcript variants for a given gene that map to an UniProt ID.

In [ ]:
#Run this code to display widget
display(EnsemblID_serch)

Textarea(value='', description='Enter Ensembl ID', placeholder='ENSEMBL ID', style=DescriptionStyle(descriptio…

In [ ]:
#Run this code to perform Ensembl search, it will take the ID from the widget
ensembl_search(EnsemblID_serch.value)

Ensembl search successful
{'ensembl_transcript': {'transcript_id': '5507800', 'seqRegionStart': 32615448, 'seqRegionEnd': 32697095, 'alignment_difference': None}, 'uniprot_accession': {'uniprot_entry': {'ac': 'A0A8I5KWH8', 'id': 'A0A8I5KWH8_HUMAN', 'uniprot_checksum': '7AA3847ABA0CC68B', 'sequence_length': 625, 'segment_start': 1, 'segment_end': 252}, 'structures': [{'summary': {'model_identifier': 'A0A8I5KWH8_1-252:3eap.1.A', 'model_category': 'TEMPLATE-BASED', 'model_url': 'https://swissmodel.expasy.org/3d-beacons/uniprot/A0A8I5KWH8.pdb?range=1-252&template=3eap.1.A&provider=swissmodel', 'model_format': 'PDB', 'model_type': 'ATOMIC', 'model_page_url': 'https://swissmodel.expasy.org/repository/uniprot/A0A8I5KWH8?range=1-252&template=3eap.1.A', 'provider': 'SWISS-MODEL', 'created': '2023-06-06', 'sequence_identity': Decimal('1.0'), 'uniprot_start': 1, 'uniprot_end': 252, 'coverage': Decimal('0.403'), 'confidence_type': 'QMEANDisCo', 'confidence_version': '4.3.1', 'confidence_avg_local_



---



##4.&nbsp; DOWNLOAD SPECIFIC MODEL

Once you found the model/ models you need, you can customise this code to download the MMCIF file to Google Drive

If multiple models are required, separated them with a comma.

In [ ]:
#Run this code to display widget
display(Uniprot_ID)
display(model_download)
print("Run the next code chunk to download the files")

NameError: ignored

In [ ]:
#@title
#Run this code to search the network using the input from the widget
structures = Search3DBeacons(Uniprot_ID.value)

### Run this code to download the models, it will use the input from the widget
for structure in structures:
  model = structure.get('summary', {}).get('model_identifier')
  if model in model_download.value:
    model_url = structure.get("summary", {}).get("model_url")
    download_file(model_url)

--2023-06-15 10:35:55--  https://www.ebi.ac.uk/pdbe/static/entry/6vqo_updated.cif
Resolving www.ebi.ac.uk (www.ebi.ac.uk)... 193.62.193.80
Connecting to www.ebi.ac.uk (www.ebi.ac.uk)|193.62.193.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2459022 (2.3M) [text/plain]
Saving to: ‘6vqo_updated.cif’

6vqo_updated.cif    100%[===================>]   2.34M   554KB/s    in 4.7s    

2023-06-15 10:36:01 (516 KB/s) - ‘6vqo_updated.cif’ saved [2459022/2459022]

--2023-06-15 10:36:01--  https://www.ebi.ac.uk/pdbe/static/entry/6xre_updated.cif
Resolving www.ebi.ac.uk (www.ebi.ac.uk)... 193.62.193.80
Connecting to www.ebi.ac.uk (www.ebi.ac.uk)|193.62.193.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7932695 (7.6M) [text/plain]
Saving to: ‘6xre_updated.cif’

6xre_updated.cif    100%[===================>]   7.56M  1.56MB/s    in 7.9s    

2023-06-15 10:36:09 (981 KB/s) - ‘6xre_updated.cif’ saved [7932695/7932695]





---



# TROUBLESHOOTING

| Problem / Response code| Possible cause | Solution |
| :---------: |  :- | :- |
| 202 | **Accepted request.** <br> 1. The sequence search submission was correct, and the job has been assigned a job identifier. <br> 2. The sequence search job is currently running.| 1.Please wait until the sequence search run completes. It can take 5-10+ minutes.
| 400 | **Bad request** <br>  1. Malformed UniProt accession.  <br>  2. Possibly invalid sequence.  <br>  3. Job identifier not found.|1. Please check that the input UniProt accession is correct. <br>  2. Please check your input sequence and retry the submission. <br> 3. Please check if your job identifier is correct.
| 404| **Not found.** <br> No results found for the given request.| 1. There may be no results for a specific UniProt accession or protein sequence
|500| **Internal Server Error** | 1. This error might be due to scheduled maintenance or rarely technical issues. <br> 2. Please try again later. If the issue persists, please email pdbekb_help@ebi.ac.uk



# Bugs

If you encounter any bugs, please report the issue to pdbekb_help@ebi.ac.uk.
